In [1]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """
    Interacts with OpenAI's API for chat completions.
    """
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        """
        Initialize with model, API key, and base URL.
        """
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """
        Generate a response from input messages.
        """
        params = {'messages': messages, 'model': self.model, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

In [2]:
class Agent:
    """
    Integrates LLM client, tools, and memory.
    """
    def __init__(self, llm_client, tools=None, memory=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = memory

    def run(self, messages: List[Dict[str, str]]):
        """
        Generates a response from the LLM client.
        """
        # Generate response using the LLM client
        response = self.llm_client.generate(messages)
        return response

In [3]:
import getpass
from openai import OpenAI

# Ask user for API key securely in Colab
OPENAI_API_KEY = getpass.getpass("Enter your API Key: ")

# Initialize the client with a custom base_url (OpenRouter in this case)
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# --- Define the Agent class ---
class Agent:
    def __init__(self, llm_client, model="openai/gpt-oss-20b"):
        self.llm_client = llm_client
        self.model = model

    def run(self, messages):
        """
        Run the agent with given messages and return the response.
        """
        response = self.llm_client.chat.completions.create(
            model=self.model,
            messages=messages
        )
        return response.choices[0].message.content


# --- Example usage ---
messages = [
    {"role": "system", "content": "You are a security assistant."},
    {"role": "user", "content": "Hey! This is Roberto!"}
]

myAgent = Agent(llm_client=client)

response = myAgent.run(messages=messages)
print(response)


Enter your API Key: ··········
Hi Roberto! 👋 How can I help you with security today?


In [4]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""

    def __init__(self):
        self.messages = []

    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)

    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)

    def add_conversation(self, user_message: Dict, assistant_message: Dict):
        """Add a user-assistant conversation."""
        self.add_messages([user_message, assistant_message])

    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()

    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

In [5]:
class Agent:
    """Integrates LLM client, tools, and memory."""

    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        """Generate a response using LLM client and store context."""
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history)
        self.memory.add_message(response)
        return response

In [11]:
import getpass
from openai import OpenAI

# Ask for OpenRouter API key securely
OPENROUTER_API_KEY = getpass.getpass("Enter your OpenRouter API Key: ")

# Initialize the client with base_url + api_key
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# --- Define the Agent class with memory ---
class Agent:
    def __init__(self, llm_client, system_message, model="openai/gpt-oss-20b"):
        self.llm_client = llm_client
        self.system_message = system_message
        self.model = model
        self.chat_history = [system_message]  # keep memory

    def run(self, user_message):
        # Add user message to history
        self.chat_history.append(user_message)

        # Call the model
        response = self.llm_client.chat.completions.create(
            model=self.model,
            messages=self.chat_history
        )

        # Extract the assistant reply
        reply = response.choices[0].message.content

        # Add assistant reply to history for memory
        self.chat_history.append({"role": "assistant", "content": reply})

        return reply


# --- Example usage ---
system_message = {"role": "system", "content": "You are a security assistant."}
agent = Agent(llm_client=client, system_message=system_message)

# First user message
user_message = {"role": "user", "content": "Hey! This is Roberto!"}
print(agent.run(user_message))

# Follow-up message (tests memory)
follow_up_message = {"role": "user", "content": "What was my name?"}
print(agent.run(follow_up_message))


Enter your OpenRouter API Key: ··········
Hey Roberto! How can I help you with security today?
Your name was Roberto.


In [12]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

In [13]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

In [14]:
def get_weather(location: str) -> str:
    """Gets weather information."""
    return f"{location}: 80F."

def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

In [15]:
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}

jump_func_dict = {
    'type': 'function',
    'function': {
        'name': 'jump',
        'description': 'Jump a specific distance.',
        'parameters': {
            'properties': {'distance': {'type': 'string'}},
            'required': ['distance'],
            'type': 'object'
        }
    }
}


In [22]:
import getpass
from openai import OpenAI

# 🔑 Enter your API key securely in Colab
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API Key: ")

# ✅ Initialize client
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# --- Define the Agent class with tools ---
class Agent:
    def __init__(self, llm_client, system_message, tools=None, model="openai/gpt-oss-20b"):
        self.llm_client = llm_client
        self.system_message = system_message
        self.model = model
        self.chat_history = [system_message]
        self.tools = tools if tools else []

    def run(self, user_message):
        # Add user message to history
        self.chat_history.append(user_message)

        # Send to LLM with optional tools
        response = self.llm_client.chat.completions.create(
            model=self.model,
            messages=self.chat_history,
            tools=self.tools if self.tools else None
        )

        # Extract assistant reply
        reply = response.choices[0].message

        # Store reply in chat history
        self.chat_history.append({"role": reply.role, "content": reply.content})

        return reply


# --- Example tools (replace with your own dicts) ---
get_weather_func_dict = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the weather for a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"}
            },
            "required": ["location"]
        }
    }
}

jump_func_dict = {
    "type": "function",
    "function": {
        "name": "jump",
        "description": "Make the assistant jump",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
}

# --- Usage ---
system_message = {"role": "system", "content": "You are a helpful assistant."}
tools = [get_weather_func_dict, jump_func_dict]

agent = Agent(llm_client=client, system_message=system_message, tools=tools, model="openai/gpt-oss-20b")

# First user query
user_message = {"role": "user", "content": "What is the weather in Virginia?"}
response = agent.run(user_message)

print(response)


Enter your OpenAI API Key: ··········
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_S72OEAtsvUmrScufyzINy8oE', function=Function(arguments='{"location":"Virginia"}', name='get_weather'), type='function', index=0)], reasoning='User wants weather in Virginia. Need to call get_weather with location "Virginia".', reasoning_details=[{'type': 'reasoning.text', 'text': 'User wants weather in Virginia. Need to call get_weather with location "Virginia".', 'format': 'unknown', 'index': 0}])


In [23]:
import json

tool_response = response.tool_calls[0]
tool_arguments = json.loads(tool_response.function.arguments)
tool_arguments

tool_execution_results = get_weather(**tool_arguments)
tool_execution_results

'Virginia: 80F.'

In [24]:
tool_message = {
    "role": "tool",
    "tool_call_id": tool_response.id,
    "name": tool_response.function.name,
    "content": str(tool_execution_results)
}

final_response = agent.run(tool_message)
final_response

ChatCompletionMessage(content='It’s currently warm in Virginia—around 80\u202f°F.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='The user asked: "What is the weather in Virginia?" The system performed a function call and got a response: Virginia: 80F. This is presumably the correct answer. But the user now expects an answer. However, the tool call returned "Virginia: 80F." We need to output a final answer. However, currently we have a non-true expected result? The function call "get_weather" returns "Virginia: 80F." The assistant should give that answer, potentially with an explanation. The user wants the weather. So answer: "It is about 80°F". Should we elaborate? Unsure. I think a simple response: "The weather in Virginia is currently around 80°F." That would satisfy.', reasoning_details=[{'type': 'reasoning.text', 'text': 'The user asked: "What is the weather in Virginia?" The system performed a function call and got a

In [25]:
# Python function
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

# Pydantic Model
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

In [26]:
# Extract tool arguments (JSON string)
tool_response = response.tool_calls[0]
tool_arguments = tool_response.function.arguments

response_model = GetWeatherSchema.model_validate_json(tool_arguments)
response_model

GetWeatherSchema(location='Virginia')

In [27]:
GetWeatherSchema.model_json_schema()

{'description': 'Get weather information based on location.',
 'properties': {'location': {'description': 'Location to get weather for',
   'title': 'Location',
   'type': 'string'}},
 'required': ['location'],
 'title': 'GetWeatherSchema',
 'type': 'object'}

In [29]:
tools: [
    {
        "type": "function",
        "function": {
            "name": "function_name",
            "description": "Function description. (optional)",
            "parameters": "The parameters the functions accepts, described as a JSON Schema object. (optional)"
        }
    }
]

In [30]:
def to_openai_function_call_definition(name: str, model: BaseModel):
    schema_dict = model.model_json_schema()
    description = schema_dict.pop("description", "")
    schema_dict.pop("title", None)  # Remove the title field to exclude the model name
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": schema_dict
        }
    }

In [31]:
function_call_definition = to_openai_function_call_definition("get_weather", GetWeatherSchema)
function_call_definition

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get weather information based on location.',
  'parameters': {'properties': {'location': {'description': 'Location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

In [32]:
from pydantic import BaseModel
from typing import Callable, Type

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

In [33]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator

In [34]:
# Python function arguments schema
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

# Convert the AgentTool to OpenAI's Function Calling format
get_weather.to_openai_function_call_definition()

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': '',
  'parameters': {'properties': {'location': {'description': 'location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

In [35]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    # Existing code...

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

In [37]:
import json

# --- Define the actual tool function ---
def get_weather(location: str):
    """Mock weather function"""
    return f"The weather in {location} is sunny and 25°C."

# --- Validate JSON string arguments ---
tool_json_arguments = '{ "location": "Virginia" }'

try:
    # Convert JSON string to dict
    tool_arguments_dict = json.loads(tool_json_arguments)

    # Run the tool with unpacked arguments
    result = get_weather(**tool_arguments_dict)

    print("✅ Tool executed successfully:", result)

except json.JSONDecodeError as e:
    print("❌ Invalid JSON:", e)
except TypeError as e:
    print("❌ Argument mismatch:", e)


✅ Tool executed successfully: The weather in Virginia is sunny and 25°C.


In [38]:
from typing import Any, Dict, List, Optional

class AgentToolExecutor:
    """Manages tool registration and execution."""

    def __init__(self, tools: Optional[List[AgentTool]] = None):
        self.tools: Dict[str, AgentTool] = {}
        if tools:
            for tool in tools:
                self.register_tool(tool)

    def register_tool(self, tool: AgentTool):
        """Registers a tool."""
        if tool.name in self.tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self.tools[tool.name] = tool

    def execute(self, tool_name: str, *args, **kwargs) -> Any:
        """Executes a tool by name with given arguments."""
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        try:
            return tool(*args, **kwargs)
        except Exception as e:
            raise ValueError(f"Error executing tool '{tool_name}': {e}") from e

    def get_tool_names(self) -> List[str]:
        """Returns a list of all registered tool names."""
        return list(self.tools.keys())

    def get_tool_details(self) -> str:
        """Returns details of all registered tools."""
        tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in self.tools.values()]
        return '\n'.join(tools_info)

In [42]:
import logging
from typing import Dict, List, Optional

logger = logging.getLogger(__name__)

class Agent:
    """Integrates LLM client, tools, memory, and manages tool executions."""

    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None

        # Register and convert tools
        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

    def run(self, user_message: Dict[str, str]):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message(user_message)

        for _ in range(self.max_iterations):
            chat_history = [self.system_message] + self.memory.get_messages() + self.tool_history
            response = self.llm_client.generate(chat_history, tools=self.function_calls)

            if self.parse_response(response):
                continue
            else:
                self.memory.add_message(response)
                self.tool_history = []
                return response

    def parse_response(self, response) -> bool:
        """Executes tool calls suggested by the LLM and updates tool history."""
        import json

        if response.tool_calls:
            self.tool_history.append(response)
            for tool in response.tool_calls:
                tool_name = tool.function.name
                tool_args = tool.function.arguments
                tool_args_dict = json.loads(tool_args)
                try:
                    logger.info(f"Executing {tool_name} with args: {tool_args}")
                    execution_results = self.executor.execute(tool_name, **tool_args_dict)
                    self.tool_history.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "name": tool_name,
                        "content": str(execution_results)
                    })
                except Exception as e:
                    raise ValueError(f"Execution error in tool '{tool_name}': {e}") from e
            return True
        return False


In [46]:
# Define the system message
system_message = {"role": "system", "content": "You are a helpful assistant."}

# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

# Run Task
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

['The weather in Virginia is sunny and 25°C.']

In [47]:
# Enable Logging
import logging
logging.basicConfig(level=logging.INFO)

# Reset previous interactions
agent.memory.reset_memory()

# Define a new task
user_message = {"role": "user", "content": "What is the weather in Virginia, Washington and New York?"}

# Generate a response using the agent
response = agent.run(user_message)
response

AttributeError: 'Agent' object has no attribute 'memory'